### Vlance Crawler

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import re

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.vlance.vn/#")

login_button = driver.find_element(By.ID, "btn-login")
login_button.click()
time.sleep(5)

username = driver.find_element(By.ID, "login_username_header")
password = driver.find_element(By.ID, "login_password_header")
username.send_keys("uyenbaekexol04@gmail.com")
password.send_keys("3107weareone")

print('Solve captcha manually...............')
time.sleep(30)

sumbit_button = driver.find_element(By.ID, "btn-submit-login-header")
sumbit_button.click()

Solve captcha manually...............


In [3]:
columns = [
    "Type", "Title", "Services", "Skills", "Description", 
    "In4_project", "In4_employment", "Num_applicants", 
    "Applicants", "Duration", "Price", "Link"
]

df = pd.DataFrame(columns=columns)

In [4]:
page = 121
while True:
    driver.get(f"https://www.vlance.vn/viec-lam-freelance/cpath_cac-cong-viec-it-va-lap-trinh_page_{page}")
    print(f"Page: {page}")
    try:
        WebDriverWait(driver, 23).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "fr-name"))
        )
    except Exception as e:
        print("Error: ", e)
        driver.quit()
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    h3_tags = soup.find_all('h3', attrs={'class':"fr-name block-title"})
    print(len(h3_tags))
    links = []
    Type = []
    Title = []
    Services = []
    Skills = []
    Description = []
    In4_project = []
    In4_employment = []
    Num_applicants = []
    Applicants = []
    Duration = []
    Price = []

    for h3 in h3_tags:
        links.append("https://www.vlance.vn" + h3.find('a')['href'])
        print(h3.find('a')['href'])
        url = f"https://www.vlance.vn{h3.find('a')['href']}"
    
        driver.get(url)
        try:
            WebDriverWait(driver, 30).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h1.title.block-title"))
            )
        except Exception as e:
            print("Error: ", e)
            driver.quit()

        
        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Hanlde case secrect project
        secret = soup.find('div', attrs={'class':"span12 btn-connect"})
        if secret:
            print("Secret project")
            Title.append(np.nan)
            Services.append(np.nan)
            Skills.append(np.nan)
            Description.append(np.nan)
            In4_project.append(np.nan)
            In4_employment.append(np.nan)
            Num_applicants.append(np.nan)
            Price.append(np.nan)
            Applicants.append(np.nan)
            Duration.append(np.nan)
            Type.append("Secret project")
            continue

        type = soup.find('span', attrs={'class':"job-progress-title"})
        type_acp_bid = soup.find('div', attrs={'class':"progress-job"})
       

        if type and type.text == "Chiến dịch hiển thị":
            Type.append("Chiến dịch hiển thị")

            title = soup.find('h1', attrs={'class':"title block-title"}).text
            Title.append(title)

            # service = soup.find('div', attrs={'class':"service-title"}).text
            service = np.nan
            Services.append(service)

            description = soup.find('div', attrs={'class':"row-fluid body-view review-text"}).text
            Description.append(description)

            # skills = soup.find('div', attrs={'class':"span10 skill"}).text
            skills = np.nan
            Skills.append(skills)

            in4_project = soup.find('div', attrs={'class':"description-job"})
            if in4_project:
                In4_project.append(in4_project.text)
            else:
                In4_project.append(np.nan)

            in4_employment = soup.find('div', attrs={'class':"info-employment"})
            if in4_employment:
                In4_employment.append(in4_employment.text)
            else:
                In4_employment.append(np.nan)


            num_applicants = soup.find('span', attrs={'class':"bid-samples"}).text
            Num_applicants.append(num_applicants)

            budget = soup.find('div', attrs={'class':"span4 client-bidding-between"}).text
            Price.append(budget)

            duration_avg = soup.find('span', attrs={'class':"duration-average"}).text
            Duration.append(duration_avg)

            applicants = np.nan
            Applicants.append(applicants)
            
        else:
            if type and type.text == "Đăng việc":
                Type.append("Đăng việc")
                
                title = soup.find('h1', attrs={'class':"title block-title"}).text
                Title.append(title)

                service = soup.find('div', attrs={'class':"service-title"})
                if service:
                    Services.append(service.text)
                else: 
                    Services.append(np.nan)

                description = soup.find('div', attrs={'class':"span10 description"}).text
                Description.append(description)

                skills = soup.find('div', attrs={'class':"span10 skill"})
                if skills:
                    Skills.append(skills.text)
                else:
                    Skills.append(np.nan)

                in4_project = soup.find('div', attrs={'class':"description-job"})
                if in4_project:
                    In4_project.append(in4_project.text)
                else:
                    In4_project.append(np.nan)

                in4_employment = soup.find('div', attrs={'class':"info-employment"})
                if in4_employment:
                    In4_employment.append(in4_employment.text)
                else:
                    In4_employment.append(np.nan)

                bid_counter = soup.find('span', attrs={'class':"bid-counter"}).text
                Num_applicants.append(bid_counter)

                bid_price = soup.find('div', attrs={'class':"span7 offset1 client-bidding-between-new"}).text
                Price.append(bid_price)

                duration_avg = soup.find('span', attrs={'class':"duration-average"}).text
                Duration.append(duration_avg)

                applicants = soup.find('div', attrs={'class':"row-fluid container list-bid-new"})
                list_applicants = []
                if applicants:
                    a_tags = applicants.find_all('h3', attrs={'class':"title"})
                    for user_href in a_tags:
                        list_applicants.append(user_href.find('a')['href'])
                Applicants.append(list_applicants)

            else:
                if type_acp_bid and re.search(r"Nhận chào giá", type_acp_bid.text.strip()):
                    print("fuck off")
                    Type.append("Nhận chào giá")
                    title = soup.find('h1', attrs={'class':"title block-title"}).text
                    Title.append(title)
                    #print(title)

                    service = soup.find('div', attrs={'class':"service-title"})
                    if service:
                        Services.append(service.text)
                    else: 
                        Services.append(np.nan)

                    description = soup.find('div', attrs={'class':"span10 description"}).text
                    Description.append(description)

                    skills = soup.find('div', attrs={'class':"span10 skill"})
                    if skills:
                        Skills.append(skills.text)
                    else:
                        Skills.append(np.nan)

                    in4_project = soup.find('div', attrs={'class':"description-job"})
                    if in4_project:
                        In4_project.append(in4_project.text)
                    else:
                        In4_project.append(np.nan)

                    in4_employment = soup.find('div', attrs={'class':"info-employment"})
                    if in4_employment:
                        In4_employment.append(in4_employment.text)
                    else:
                        In4_employment.append(np.nan)

                    bid_counter = soup.find('span', attrs={'class':"bid-counter"}).text
                    Num_applicants.append(bid_counter)

                    bid_price = soup.find('div', attrs={'class':"span6 client-bidding-between"}).text
                    Price.append(bid_price)

                    duration_avg = soup.find('span', attrs={'class':"duration-average"}).text
                    Duration.append(duration_avg)

                    applicants = soup.find_all('div', attrs={'class':"profile-job-left-bottom"})
                    list_applicants = []
                    if applicants:
                        for user_href in applicants:
                            list_applicants.append(user_href.find('a')['href'])
                    Applicants.append(list_applicants)


                else:
                    print("So sick and tired")
                    Title.append(np.nan)
                    Services.append(np.nan)
                    Skills.append(np.nan)
                    Description.append(np.nan)
                    In4_project.append(np.nan)
                    In4_employment.append(np.nan)
                    Num_applicants.append(np.nan)
                    Price.append(np.nan)
                    Applicants.append(np.nan)
                    Duration.append(np.nan)
                    Type.append(np.nan)
    
    one_page = pd.DataFrame({
            "Type": Type,
            "Title": Title,
            "Services": Services,
            "Skills": Skills,
            "Description": Description,
            "In4_project": In4_project,
            "In4_employment": In4_employment,
            "Num_applicants": Num_applicants,
            "Applicants": Applicants,
            "Duration": Duration,
            "Price": Price,
            "Link": links
        })

    df = pd.concat([df, one_page], ignore_index=True)
    df.to_csv(f"job_posts_{page}.csv", index=False)

    page += 1
    if page == 234:
        break


Page: 121
10
/viec-freelance/freelancer-thiet-ke-giao-dien-web-app
/viec-freelance/download-tin-nhan-messenger-1
/viec-freelance/freelance-shopify-developer-1
/viec-freelance/hien-thi-file-excel-pdf-luu-trong-sharepoint-len-website-wordpress
/viec-freelance/tuyen-ban-da-tung-seo-tu-khoa-len-top-de-viet-bai-va-seo-cho-nhieu-chu-de-1
/viec-freelance/test-kiem-thu-khep-kin-android
/viec-freelance/them-goi-y-tren-text-editor-bang-summernote
/viec-freelance/can-gap-nguoi-test-cho-he-thong
fuck off
/viec-freelance/cao-du-lieu-review-tren-lazada
/viec-freelance/login-zalo-ios
Page: 122
10
/viec-lam/viet-plugin-check-thong-tin-san-pham
/viec-freelance/bo-sung-tinh-nang-cho-web-wordpress
/viec-freelance/freelancer-viet-mobile-app-no-code
/viec-freelance/freelance-lap-trinh-mobile-app-no-code
/viec-lam/tuyen-dung-sinh-vien-freelancer-tai-vinh-nghe-an-du-an-react-js-next-js-va-node-js
/viec-freelance/can-hoan-thien-website-hien-tai-vuejs-1
/viec-freelance/them-hieu-ung-cho-website-cong-ty
/viec-f

TimeoutException: Message: timeout: Timed out receiving message from renderer: 286.237
  (Session info: chrome=129.0.6668.103)
Stacktrace:
	GetHandleVerifier [0x00007FF6DFACB095+29557]
	(No symbol) [0x00007FF6DFA3FA50]
	(No symbol) [0x00007FF6DF8FB56A]
	(No symbol) [0x00007FF6DF8E41E7]
	(No symbol) [0x00007FF6DF8E3EEA]
	(No symbol) [0x00007FF6DF8E1D65]
	(No symbol) [0x00007FF6DF8E25BF]
	(No symbol) [0x00007FF6DF8F0F4E]
	(No symbol) [0x00007FF6DF909A1D]
	(No symbol) [0x00007FF6DF90F5BA]
	(No symbol) [0x00007FF6DF8E2D24]
	(No symbol) [0x00007FF6DF909879]
	(No symbol) [0x00007FF6DF998861]
	(No symbol) [0x00007FF6DF976F33]
	(No symbol) [0x00007FF6DF94116F]
	(No symbol) [0x00007FF6DF9422D1]
	GetHandleVerifier [0x00007FF6DFDFC96D+3378253]
	GetHandleVerifier [0x00007FF6DFE48497+3688311]
	GetHandleVerifier [0x00007FF6DFE3D1CB+3642539]
	GetHandleVerifier [0x00007FF6DFB8A6B6+813462]
	(No symbol) [0x00007FF6DFA4AB5F]
	(No symbol) [0x00007FF6DFA46B74]
	(No symbol) [0x00007FF6DFA46D10]
	(No symbol)

In [11]:
print("Length of Type:", len(Type))
print("Length of Title:", len(Title))
print("Length of Services:", len(Services))
print("Length of Skills:", len(Skills))
print("Length of Description:", len(Description))
print("Length of In4_project:", len(In4_project))
print("Length of In4_employment:", len(In4_employment))
print("Length of Num_applicants:", len(Num_applicants))
print("Length of Applicants:", len(Applicants))
print("Length of Duration:", len(Duration))
print("Length of Price:", len(Price))
Type

Length of Type: 10
Length of Title: 10
Length of Services: 10
Length of Skills: 10
Length of Description: 10
Length of In4_project: 10
Length of In4_employment: 10
Length of Num_applicants: 10
Length of Applicants: 10
Length of Duration: 10
Length of Price: 10


['Đăng việc',
 'Đăng việc',
 'Đăng việc',
 'Đăng việc',
 'Đăng việc',
 'Đăng việc',
 'Đăng việc',
 'Nhận chào giá',
 nan,
 'Đăng việc']

In [13]:
df.head(10)

,Type,Title,Services,Skills,Description,In4_project,In4_employment,Num_applicants,Applicants,Duration,Price,Link
0,Nhận chào giá,\nThiết kế tool tự động điền captcha khi đăng ...,\nDịch vụ cần thuê:\n\n\nViết phần mềm theo yê...,NaN,"\n\nChào các bạn, \nHiện tại mình đang cần thi...",\n\nID dự án\n37869\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ph...,\n Chào giá: ...,[/freelancer/tuan-thanh-19],\n Trung bình: 3...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/thiet-ke-...
1,Nhận chào giá,\nThiết kế web bán khoá theo mẫu có sẵn Hết hạ...,\nDịch vụ cần thuê:\n\n\nDựng website bán hàng...,"\nKỹ năng\n\nSEO , \nNhập dữ liệu , \nThiết kế...",\n\nThiết kế web bán khoá điện tử \nCần bạn nà...,\n\nID dự án\n37865\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Na...,\n Chào giá: ...,"[/freelancer/thanh-tien, /freelancer/nghiem-qu...",\n Trung bình: 7...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/thiet-ke-...
2,Nhận chào giá,\nSet up proxy 3g Hết hạn nhận chào giá\n,\nDịch vụ cần thuê:\n\n\nViết phần mềm theo yê...,"\nKỹ năng\n\nLinux , \nAndroid , \nPython",\n\nTạo hệ thống tự động tạo proxy từ usb 3g/4...,\n\nID dự án\n37864\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Wa...,\n Chào giá: ...,[/freelancer/tran-5],\n Trung bình: 3...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/set-up-pr...
3,Nhận chào giá,\nDesigner cho project vuejs Hết hạn nhận chào...,\nDịch vụ cần thuê:\n\n\nViết phần mềm theo yê...,"\nKỹ năng\n\nBootstrap , \nresponsive",\n\n- Chỉnh lại màu của cả website cho hài hòa...,\n\nID dự án\n37861\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Th...,\n Chào giá: ...,"[/freelancer/dottp-tech, /freelancer/pham-van-...",\n Trung bình: 4...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/designer-...
4,Nhận chào giá,\nViết phần mềm và web cho doanh nghiệp Hết hạ...,\nDịch vụ cần thuê:\n\n\nViết phần mềm theo yê...,NaN,\n\nLàm việc theo giờ hoặc theo dự án đều được...,\n\nID dự án\n37860\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n BA...,\n Chào giá: ...,"[/freelancer/vien-nguyen-9, /freelancer/hoang-...",\n Trung bình: 1...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/viet-phan...
5,Nhận chào giá,\nLiệt kê theo điều kiện trên exel dùng VBA Đã...,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,\nKỹ năng\n\nVBA,\n\nMình có một file excel cần xữ lý dữ liệu n...,\n\nID dự án\n37859\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n ph...,\n Chào giá: ...,"[/freelancer/le-thanh-phuoc-1, /freelancer/pho...",\n Trung bình: 4...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/liet-ke-t...
6,Nhận chào giá,\n10 cộng tác viên sử dụng ứng dụng Android tr...,\nDịch vụ cần thuê:\n\n\nTest và kiểm tra lỗi ...,"\nKỹ năng\n\nTiếng Anh , \nAndroid",\n\nĐiều kiện: \nSmart phone chạy Android 6.0 ...,\n\nID dự án\n37855\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Lo...,\n Chào giá: ...,"[/freelancer/hung-doan-minh, /freelancer/thuy-...",\n Trung bình: 2...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/10-cong-t...
7,Nhận chào giá,\nNâng cấp giao diện trang web joomla Hết hạn ...,\nDịch vụ cần thuê:\n\n\nThiết kế giao diện we...,"\nKỹ năng\n\nJoomla , \nPHP",\n\n-Chỉnh sửa lại giao diện trang home : www....,\n\nID dự án\n37854\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Ng...,\n Chào giá: ...,"[/freelancer/nguyen-trung-23, /freelancer/nam-...",\n Trung bình: 6...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/nang-cap-...
8,Nhận chào giá,\nThiết kế app tính toán lô đề theo công thức ...,\nDịch vụ cần thuê:\n\n\nLàm Mobile App theo y...,"\nKỹ năng\n\nTrình bày , \nLập trình ứng dụng",\n\nỨng dụng di động cho phép tính toán lô đề ...,\n\nID dự án\n37852\nNgày đăng\n\n ...,\n\n\n\n\n\n\n\n\n\n Xu...,\n Chào giá: ...,[],\n Trung bình: 0...,\n\n Thấp nhất: ...,https://www.vlance.vn/viec-freelance/thiet-ke-...
9,Nhận chào giá,\nĐánh dấu dòng và cột của ô trong excel khi d...,\nDịch vụ cần thuê:\n\n\nViết phần mềm theo yê...,"\nKỹ năng\n\nVBA , \nMS Excel",\n\nYêu cầu : \n- Tạo Addin đánh dấu dòng và c...,\n\n

In [14]:
df.to_csv("job_posts.csv", index=False)

## Từ khúc sau này là code test crawl freelancer

In [16]:
url = "https://www.vlance.vn/viec-freelance/long-hinh-anh-quy-hoach-vao-google-map"


In [28]:
print(title)
print(service)
print(skills)
print(description)
print(in4_project)
print(in4_employment)
print(bid_counter)
print(bid_price)
print(duration_avg)
list_applicants


Lồng hình ảnh quy hoạch vào google map Hết hạn nhận chào giá


Dịch vụ cần thuê:


Phát triển ứng dụng web 



Kỹ năng

Bất động sản  


Tôi có 1 hình ảnh của 1 dự án bất động sản đang triển khai, bao gồm các lô đất được phân trên hình ảnh đó, hãy giúp tôi lồng nó vào google map để người dùng dễ hình dung được là lô nào ở đâu trên google map.




ID dự án
79847
Ngày đăng

                                            07/10/2024, 17:11                                    
Chỉ còn
Hết hạn nhận chào giá
Địa điểm

                Hà Nội            
Ngân sách

                5.000.000đ                                                             - 20.000.000đ                                                
Hình thức làm việc

                                                                Làm online                                                        
Hình thức trả lương

                                    Trả theo dự án                            












                        Thuy

['/freelancer/nguyen-nhat-hao-1',
 '/freelancer/cong-ty-phan-mem-nextcore',
 '/freelancer/tuan-anh-truong-10',
 '/freelancer/pm-hung']

In [30]:
url = "https://www.vlance.vn/freelancer/pm-hung"
driver.get(url)

In [31]:
try:
    WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.id_profile"))
    )
except Exception as e:
    print("Error: ", e)
    driver.quit()

In [32]:
try:
    see_more_skill = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "see-more-skill"))
    )
    see_more_skill.click()
    print("Clicked on 'see-more-skill'")
except Exception as e:
    print("Element 'see-more-skill' not found or not clickable:", e)

Clicked on 'see-more-skill'


In [33]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [34]:
id = soup.find('div', attrs={'class':"id_profile"}).text
name = soup.find('span', attrs={'itemprop':"name"}).text
title = soup.find('span', attrs={'itemprop':"title editable tf300"}).text
region = soup.find('div', attrs={'class':"regional-price"}).text
overview = soup.find('div', attrs={'class':"overview overview-mobile"}).text
services = soup.find('div', attrs={'class':"service-freelancer"}).text
summary = soup.find('div', attrs={'class':"summary-profile profile-rate summary-title summary-block"}).text
summary_working = soup.find('div', attrs={'class':"summary-profile profile-rate summary-working"}).text


In [35]:
print(id)
print(name)
print(title)
print(region)
print(overview)
print(services)
print(summary)
print(summary_working)


id. 583721 
Freelancer

Seo, Sem profession, Marketing leader 






Hà Nội 




Giới thiệu 

Tôi hiện mong muốn được làm freelancer để cải thiện thêm kĩ năng và tăng thu nhập. Thế mạnh của tôi là Seo web, tối ưu web, chạy quảng cáo Facebook, google, tiktok, zalo...
Tôi có ít nhất 10 năm kinh nghiêm trong các lĩnh vực này, và liên tụ trau dồi thêm các kĩ năng mới ở các nền tảng trẻ như Tiktok, zalo...
Thành tính của tôi bên cạnh việc SEO từ khóa từ top 8 đến top 3, tôi cũng đã giúp công ty mình có doanh thu ít nhất 1 tỷ / tháng dựa trên mảng digital tôi quản lý.
Tôi có kinh nghiệm quản lý ngân sách từ 2-350 triệu vnđ/ tháng và có thể phân bổ ngân sách lớn.
Bất cứ chiến dịch nào tôi thực hiện đều dặt mục tiêu đem lại chuyển đổi và doanh số cao nhất bên cạnh các mục tiêu thương hiệu khác.
Tôi cả đã pass qua các chứng chỉ của google như:
Ngoại ngữ : Toeic 875, Vstep bậc 4
Shopping ads Certification
Google Ads Video Certification
Google Ads Search Certification
Google Ads Display Certific